In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
#crate paths for csv files
CURR_DIR = os.getcwd()
print(CURR_DIR)
zlecenia_path = os.path.join(CURR_DIR, 'Zlecenia.csv')
godziny_path = os.path.join(CURR_DIR, 'Godziny.csv')

C:\Users\ksobo\Desktop\DS_final_project


In [3]:
#Zaciagniecie baz danych
obciazenie_df = pd.read_csv(zlecenia_path, sep=';')
godziny_df = pd.read_csv(godziny_path, sep=';')

In [4]:
#Ekstrakcja danych i organizacja danych - pozostawienie tylko kolumn wymaganych, pozostawienie tylko projektów "w toku" - nie działa
obciazenie_df1 = obciazenie_df.drop(columns=['work_preparation_Plan','work_preparation_done'], axis=1)
obciazenie_df2 = obciazenie_df1.loc[obciazenie_df1['Active'].isin(['Active'])]

In [5]:
#Left join po No z godzinami
obciazenie_df2 = pd.merge(obciazenie_df2, godziny_df, how='left', on='No')

#TEST!!!!! wypelnienie NaN zerami
#obciazenie_df2['Wszystkie.technol'] = obciazenie_df2['Wszystkie.technol'].fillna(0)
#obciazenie_df2['Wszystkie.wykonane'] = obciazenie_df2['Wszystkie.wykonane'].fillna(0)

In [6]:
#Zdefiniowanie tygodnia obecnego
#przyjmujemy obecny jako 202112(tydzień zaciągnięcia danych),
#docelowo może zostać zamieniony funkcją obliczającą jak poniżej
#WPISAĆ FUNKCJĘ!!!!!!!!!

current_week = 202112


,No,Description,Weight_for_1,Quantity,Typ,Finnish_week,Active,Pre_fabrication_plan,Wszystkie.technol,Wszystkie.wykonane,Montaz.technol,Montaz.wykonane,Spawanie.technol,Spawanie.wykonane,Szlifowanie.technol,Szlifowanie.wykonane,Pozostale.technol,Pozostale.wykonane
0,191,3130,6400,1,B,202111,Active,201848,380.0,424.5,126.0,124.0,174.0,155.0,56.0,85.0,24.0,60.5
1,192,3130,6400,1,B,202123,Active,201907,423.5,507.5,140.0,165.0,177.0,197.5,70.0,61.0,36.5,84.0
2,870,2016,2420,2,C,202146,Active,202030,372.0,462.5,142.0,153.5,104.0,132.0,40.0,60.5,86.0,116.5
3,908,2016,2420,2,C,202146,Active,202038,372.0,116.0,142.0,9.0,104.0,4.0,40.0,4.0,86.0,99.0
4,1015,1360,890,2,A,202008,Active,202001,225.2,160.5,64.0,62.5,68.0,62.0,32.0,1.0,61.2,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,1268,3230,6400,1,B,202234,Active,202218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,1269,3230,6400,1,B,202244,Active,202236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,1270,3230,6400,1,B,202307,Active,202249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,1271,3230,6400,1,B,202319,Active,202311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#Zmiana terminu rozpoczęcia oraz zakończenia na obecny jeżeli miało mieć ono miejsce w przeszłości
obciazenie_df2.loc[obciazenie_df2['Finnish_week'] < current_week, 'Finnish_week'] = current_week

obciazenie_df2.loc[obciazenie_df2['Pre_fabrication_plan'] < current_week, 'Pre_fabrication_plan'] = current_week

In [8]:
#Zastapienie NaN w godzinach wartościami przewidzianymi z prekalkulacji

#A - dziób(wędka)
#Labor rate	130 rbg/1000kg
#Assembling	0,4
#Welding	0,33
#Grindining	0,07
#Else		0,2

#B - belka(środek)
#Labor rate	100 rbg/1000kg
#Assembling	0,3
#Welding	0,4
#Grindining	0,2
#Else		0,1

#C - końcóka(środek)
#Labor rate	120 rbg/1000kg
#Assembling	0,25
#Welding	0,4
#Grindining	0,2
#Else		0,15

# IF Wszystkie technol = NaN -> Weight for 1 * Quantity * Typ (IF A - 0,13 elif B - 0,1 elif C - 0,12)

In [9]:
#Obliczenie kolumn - pozostałego czasu na produkcję oraz pozostałych godzin
obciazenie_df2['Weeks_left'] = (obciazenie_df2['Finnish_week']) - (obciazenie_df2['Pre_fabrication_plan'])

#Jeżeli >52 to należy odjąc od wyniku 48 (100 - (ilość tyg w danym roku - def 52))
#Może tak się stać dla projektów na przełomie roku 202206-202146 = 60!
obciazenie_df2.loc[obciazenie_df2['Weeks_left'] > 52, 'Weeks_left'] = obciazenie_df2['Weeks_left'] - 48

#Jeżeli zlecenie jest opóźnione i wynik pozostałego czasu będzie wynosił 0
#to zostanie on doliczony do obecnego tygodnia pozdzielony przez 1
obciazenie_df2.loc[obciazenie_df2['Weeks_left'] < 1, 'Weeks_left'] = 1

#funkcja if niestety nie znalazła odpowiedniego zastosowania - loc jest polecany jako dobra praktyka

In [10]:
#???????Czy liczyć nadwyżkę 20% jako zabezpieczenie przy przekroczeniach??????
obciazenie_df2['Work_h_left'] = (obciazenie_df2['Wszystkie.technol']) - (obciazenie_df2['Wszystkie.wykonane'])

#Jeżeli zlecenie zlecenie przekroczyło godziny praca liczona jest w nadgodzinach - przyjmuje się 0 do obciażenia
obciazenie_df2.loc[obciazenie_df2['Work_h_left'] < 0, 'Work_h_left'] = 0

In [11]:
#Podzielenie pozostałej pracy równomiernie na tygodnie
obciazenie_df2['Workload_per_week'] = (obciazenie_df2['Work_h_left']) / (obciazenie_df2['Weeks_left'])
obciazenie_df2['Workload_per_week']

0      0.000000
1      0.000000
2      0.000000
3      7.529412
4     64.700000
        ...    
58          NaN
59          NaN
60          NaN
61          NaN
62          NaN
Name: Workload_per_week, Length: 63, dtype: float64

In [12]:
#Podział obciążenia na tygodnie obecny+13 tyg
obciazenie_df3 = obciazenie_df2[["No", "Workload_per_week"]]

x = current_week
for i in range(0,13):
    obciazenie_df3[x+i]=0

obciazenie_df3

<ipython-input-12-746ae2e974f9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obciazenie_df3[x+i]=0


,No,Workload_per_week,202112,202113,202114,202115,202116,202117,202118,202119,202120,202121,202122,202123,202124
0,191,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0
1,192,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0
2,870,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0
3,908,7.529412,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1015,64.700000,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,1268,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
59,1269,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
60,1270,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
61,1271,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
#Dodanie obciazenia zarezerwowanego na dany tydzien w zależności od rozpoczecia i zakonczenia danego zlecenia
#If prefabrication plan is >column index & <= finnish week -> column index.No zlecenia = Workload_per_week

In [14]:
#Zrobienie wykresu